<a href="https://colab.research.google.com/github/LiamAMcNamara/HMMs-Lab3/blob/master/McNamara_Lab_3_HMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CSE 5522 Lab #3: Hidden Markov Models**

Liam McNamara.308

June 11, 2020

---
---

**Part 1 (50 Points):** Implement the Viterbi Algorithm for HMMs for Eisner's Ice Cream Problem (predict whether each day is hot or cold based on the number of ice creams eaten). See elsner.hmm.xls for a bit more discussion on the problem. Remeber that the Viterbi algorithm computes the most likely sequence for an input.

Your solution should be able to handle variable length sequences (in the range of 3-5).

There are 3 files provided for you that have observation/emission probabilities, transition probabilities, and test sequence data (for evaluation), respectively.

Please read the probabilities and observations from their Carmen URLs, *do not hard-code the values*. Refer back to lab 2 for exampes of working with CSV files and how to download them from URLs.

The observation and transition probabilities have rows being the variable of interest, and columns being the conditioning variables. For example, P(2|H) is in the 3rd row (including header), 3rd column (including row label). The columns sum up to 1.

The test data has one line per sequence. When a sequence is less than five observations long, the last  columns are filled with zeros.



Set up the environment

In [202]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Read in the provided data from Carmen URL

In [302]:
emissionURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/observationProbs.csv'
emission_dataframe = pd.read_csv(emissionURL)
transitionURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/transitionProbs.csv'
transition_dataframe = pd.read_csv(transitionURL)
testDataURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/testData.csv'
testData_dataframe = pd.read_csv(testDataURL)

In [308]:

emissionURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/observationProbsLecture.csv'
emission_dataframe = pd.read_csv(emissionURL)
transitionURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/transitionProbsLecture.csv'
transition_dataframe = pd.read_csv(transitionURL)
testDataURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/testDataLecture.csv'
testData_dataframe = pd.read_csv(testDataURL)

Print out the top of the dataframes to make sure the data loaded correctly. The data tables should have the following format:

Emission Probabilities [3, 3]: 3 columns (P(x|...), C, H), and 3 rows corresponding the the number of ice creams

Transition Probabilities [3, 4]: 4 columns (P(x|...), C, H, START), and 3 rows (C, H, STOP)

Test Sequence Data [10, 6]: 6 columns (SeqNumber, Obs1...5), and 10 rows corresponing the the sequence sample number.

Note: The test data has one line per sequence. When a sequence is less than five observations long, the last columns are filled with zeros.

In [309]:
print("Emission Probabilities")
display(emission_dataframe.shape)
display(emission_dataframe.head())
print('\n')
print("Transition Probabilities")
display(transition_dataframe.shape)
display(transition_dataframe.head())
print('\n')
print("Test Sequence Data")
display(testData_dataframe.shape)
display(testData_dataframe.head())

Emission Probabilities


(2, 4)

,P(x|...),Sunny,Rainy,Foogy
0,No Umb.,0.9,0.2,0.7
1,Umbrella,0.1,0.8,0.3




Transition Probabilities


(4, 5)

,P(x|...),Sunny,Rainy,Foggy,START
0,Sunny,0.8,0.05,0.15,0.40
1,Rainy,0.2,0.60,0.20,0.32
2,Foggy,0.2,0.30,0.50,0.28
3,STOP,1.0,1.00,1.00,0.00




Test Sequence Data


(1, 6)

,SeqNumber,Obs1,Obs2,Obs3,Obs4,Obs5
0,first,1,1,1,2,2


Initialize matrix variables and shorthand for emission and transition arrays

In [310]:
# M.shape = [sequence length, # of hidden variables]
m_matrix = np.zeros((testData_dataframe.shape[1]-1, emission_dataframe.shape[1]-1)) 
# In this case, Sequence of length 5 with 2 possible hidden vairable values (H, C)

# C.shape = [sequence length - 1, # hidden varialbes]
c_matrix = np.delete(m_matrix, 0, 0)
# In this case, Sequence of length 5-1 with 2 hidden variable values
c_matrix = c_matrix.transpose()

# In our class notes we referred to the transition matrix as 'A'
# and the emission/observation matrix as 'B'. So I will be doing the same.
A = transition_dataframe.to_numpy()
A = np.delete(A, 0, 1)  # remove extra (label) column included in the dataframe
                        # this column included (C, H, STOP)
B = np.array(emission_dataframe)
B = np.delete(B, 0, 1)  # remove extra (label) column included in the dataframe
                        # this column included (1, 2 , 3)

sequences = testData_dataframe.to_numpy()
sequences = np.delete(sequences, 0, 1)  # remove extra (label) column included in
                                        # the df. This column inc. (1 ... n seqs)

print(A)
print(B)

[[0.8 0.05 0.15 0.4]
 [0.2 0.6 0.2 0.32]
 [0.2 0.3 0.5 0.28]
 [1.0 1.0 1.0 0.0]]
[[0.9 0.2 0.7]
 [0.1 0.8 0.3]]


In [311]:
seq = sequences[0]
display(seq)
seq = seq - 1 #decrement seq for proper indexing

array([1, 1, 1, 2, 2], dtype=object)

Step 1: Initialize

In [312]:
#print("seq:", seq)
# use priors to set first row of M matrix
for i in range(B.shape[1]):
  m_matrix[0,i] = A[i,A.shape[1]-1] * B[seq[0], i]

#normalizeRow(m_matrix, 0);
vect = np.zeros(3)
for s in range(1, len(seq)): #TODO: len(seq)
  for k in range(B.shape[1]):
    print(B[seq[s], k], "* ?") #correct B
    
    for j in range(B.shape[1]):
      #print(A[j][k])
      #print(m_matrix[s-1][j])
      #print()
      vect[j] = (A[j][k] * m_matrix[s-1][j])
      #a = np.where(v == np.max(v))
      #print(a[0][0])
    print(vect)
    max = np.max(vect)
    m_matrix[s][k] = B[seq[s], k] * max
    c = np.where(vect == max)
    c_matrix[k][s-1]=c[0][0]+1
  #normalizeRow(m_matrix, s)


0.9 * ?
[0.288  0.0128 0.0392]
0.2 * ?
[0.018  0.0384 0.0588]
0.7 * ?
[0.054  0.0128 0.098 ]
0.9 * ?
[0.20736  0.002352 0.01372 ]
0.2 * ?
[0.01296  0.007056 0.02058 ]
0.7 * ?
[0.03888  0.002352 0.0343  ]
0.1 * ?
[0.1492992 0.0008232 0.0054432]
0.8 * ?
[0.0093312 0.0024696 0.0081648]
0.3 * ?
[0.0279936 0.0008232 0.013608 ]
0.1 * ?
[0.01194394 0.00149299 0.00167962]
0.8 * ?
[0.0007465  0.00447898 0.00251942]
0.3 * ?
[0.00223949 0.00149299 0.00419904]


In [313]:
print(m_matrix)
print(c_matrix.transpose())

[[0.36       0.064      0.196     ]
 [0.2592     0.01176    0.0686    ]
 [0.186624   0.004116   0.027216  ]
 [0.01492992 0.00746496 0.00839808]
 [0.00119439 0.00358318 0.00125971]]
[[1. 3. 3.]
 [1. 3. 1.]
 [1. 1. 1.]
 [1. 2. 3.]]


In [210]:
def normalizeRow(m, i):
  #python uses pass by reference so this will change m
  norm_m = m
  row = m[i]
  row = row / np.sum(row)
  norm_m[i] = row